In [ ]:
import pickle
import os

# path = "/home/user/Datasets/interpolated/preprocess_results_10s/test"
# name = "2_480100_490100_927ZS.pkl"
# filepath = os.path.join(path, name)
# f = open(filepath, 'rb')
# data = pickle.load(f, encoding="latin1")

In [2]:
# data.keys()

dict_keys(['shapes', 'idx', 'has_preds', 'gt_preds', 'steps', 'graph', 'trajs', 'ctrs', 'theta', 'rot', 'feats', 'orig'])

In [6]:
# print(data['graph'].keys())
# print(data['graph']['feats'][0].shape)
# print(data['graph']['left_pairs'][0].shape)

In [7]:
import torch
from torch.utils.data import DataLoader, Sampler
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm
import sys
from importlib import import_module
from utils import Logger, load_pretrain, gpu, to_long,  Optimizer, StepLR

from data import InteDataset

root_path = '/home/user/Projects/LaneGCN_modified'
sys.path.insert(0, root_path)

model = import_module("lanegcn")
config, _, collate_fn, net, loss, post_process, opt = model.get_model(80)



ckpt_path = "/home/user/Projects/LaneGCN_modified/results/interpolated_map_results/lanegcn_10s/36.000.ckpt"
# ckpt_path = "/media/drl/datas/zyk/LaneGCN/results/lanegcn/17.000.ckpt"

if not os.path.isabs(ckpt_path):
    ckpt_path = os.path.join(config["save_dir"], ckpt_path)
ckpt = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
load_pretrain(net, ckpt["state_dict"])
net.eval()





dataset = InteDataset('/home/user/Datasets/interpolated/preprocess_results_10s/tmp/')
data_loader = DataLoader(
    dataset,
    batch_size=config["val_batch_size"],
    num_workers=config["val_workers"],
    collate_fn=collate_fn,
    shuffle=True,
    pin_memory=True,
)

# begin inference
preds = {}
gts = {}
cities = {}

data_list = []
for ii, data in tqdm(enumerate(data_loader)):
    data = dict(data)
    with torch.no_grad():
        output = net(data)
        results = [x.detach().cpu().numpy() for x in output["reg"]]


1it [00:01,  1.69s/it]


In [8]:
print(data['graph'][0].keys())
print(data['graph'][0]['feats'].shape)
print(data['graph'][0]['left_pairs'].shape)

dict_keys(['pre', 'lane_idcs', 'left_pairs', 'suc', 'road_polygon', 'num_nodes', 'pre_pairs', 'ctrs', 'suc_pairs', 'right_pairs', 'feats', 'left', 'right', 'idx'])
torch.Size([1214, 2])
torch.Size([10, 2])


In [9]:
from lanegcn import graph_gather, actor_gather

actors, actor_idcs = actor_gather(gpu(data["feats"]))
actor_ctrs = gpu(data["ctrs"])
actors = net.actor_net(actors)

# construct map features
graph = graph_gather(to_long(gpu(data["graph"])))
nodes, node_idcs, node_ctrs = net.map_net(graph)

# actor-map fusion cycle 
nodes = net.a2m(nodes, graph, actors, actor_idcs, actor_ctrs)
nodes = net.m2m(nodes, graph)
actors = net.m2a(actors, actor_idcs, actor_ctrs, nodes, node_idcs, node_ctrs)
actors = net.a2a(actors, actor_idcs, actor_ctrs)

In [12]:
print('len', len(node_ctrs))
print('node_ctrs[0]', node_ctrs[0].cpu().numpy().shape)
print(graph['pre'][0]['u'].cpu().numpy())

all_ctrs = torch.cat(node_ctrs, 0).cpu().numpy()
print(all_ctrs.shape)


len 6
node_ctrs[0] (1214, 2)
[   1    2    3 ... 7283 7271 7271]
(7284, 2)


In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1, figsize=[20,20])
pt = 0
mp = node_ctrs[0].cpu().numpy()

X = []
Y = []

for i in range(6):
    u = graph['pre'][i]['u'].cpu().numpy()
    print('u size:', u.shape)
    v = graph['pre'][i]['v'].cpu().numpy()
    X.append(all_ctrs[v][pt, 0])
    Y.append(all_ctrs[v][pt, 1])
    
plt.plot(mp[:, 0], node_ctrs[0][:, 1], 'r.')

# plt.plot(all_ctrs[u][:, 0], all_ctrs[u][:, 1], 'r.')

# cl = ['red', 'orange', 'yellow', 'green', 'blue', 'purple']
# plt.scatter(X, Y, s = 100, c = [cl[i] for i in range(len(X))])
# print(X, Y)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[20,20])

u = graph['pre'][i]['u'].cpu().numpy()
print('u size:', u.shape)
v = graph['pre'][i]['v'].cpu().numpy()